In [57]:
from azure.identity import AzureCliCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# Use Azure CLI credential (since you're logged in via `az login`)
credential = AzureCliCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id="6bd1f99e-e7cb-4226-b9d5-09433d793bda",
    resource_group_name="shafi1",
    workspace_name="shafi1"
)




Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [18]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
model = Model(
    path="./GBM_model_python_1749296476765_1.zip",  # Local model path
    type=AssetTypes.CUSTOM_MODEL,
    name="h2o-model",
    description="My custom model registered from local file"
)
registered_model = ml_client.models.create_or_update(model)
print(f"Model registered: {registered_model.name}, Version: {registered_model.version}")

Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Uploading GBM_model_python_1749296476765_1.zip (< 1 MB): 100%|██████████| 108k/108k [00:00<00:00, 1.15MB/s]


Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence

Model registered: h2o-model, Version: 1


In [ ]:
# model_name = "my_model"

# models = list(ml_client.models.list(name=model_name))
# print(f"\nAll registered versions of '{model_name}':")
# for m in models:
#     print(f"  - Version {m.version}: asset ID {m.id}")
    
# latest = max(models, key=lambda x: int(x.version))
# print(f"\n✅ Latest version is {latest.version}")


All registered versions of 'my_model':
  - Version 2: asset ID /subscriptions/6fab868f-e7ff-4000-b366-b5d0a27cc581/resourceGroups/iat/providers/Microsoft.MachineLearningServices/workspaces/shafi4/models/my_model/versions/2
  - Version 1: asset ID /subscriptions/6fab868f-e7ff-4000-b366-b5d0a27cc581/resourceGroups/iat/providers/Microsoft.MachineLearningServices/workspaces/shafi4/models/my_model/versions/1

✅ Latest version is 2


In [2]:
pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o


Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
from azure.ai.ml.entities import Environment
name="h2o-env"


env = Environment(
    name=name,
    version="2",  # bump version manually
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04"
)
ml_client.environments.create_or_update(env)


ml_client.environments.create_or_update(env)
# 1️⃣ Fetch all versions of the environment

env_versions = list(ml_client.environments.list(name=name))

print(f"Registered versions for environment '{name}':")
for env in env_versions:
    print(f" • Version {env.version}")

# 2️⃣ Find the latest version by numeric comparison
latest_env = max(env_versions, key=lambda e: int(e.version))
print(f"\n✅ Latest version is {latest_env.version}")


Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


Registered versions for environment 'h2o-env':
 • Version 2
 • Version 1
 • Version 3

✅ Latest version is 3


In [4]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "shafi"

In [5]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

endpoint_result = ml_client.begin_create_or_update(endpoint).result()

print(
    f"Endpint {endpoint_result.name} provisioning state: {endpoint_result.provisioning_state}"
)

Endpint shafi provisioning state: Succeeded


In [6]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpint "shafi" with provisioning state "Succeeded" is retrieved


In [28]:
from azure.ai.ml.entities import Model


model = ml_client.models.get(name="h2o-model", version="1")

Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


In [30]:

from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    Environment,
    CodeConfiguration
)
code_config = CodeConfiguration(
    code=".",              # Folder containing score.py
    scoring_script="score.py"  # Your scoring script
)

# Step 3: Attach environment and code config to deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment="h2o-env:3",
    code_configuration=code_config,
    instance_type="Standard_E2s_v3",
    instance_count=1,
)

In [33]:
# Create the deployment
blue_deployment_results = ml_client.online_deployments.begin_create_or_update(
    blue_deployment
).result()

print(f"Deployment state: {blue_deployment_results.provisioning_state}")

Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Check: endpoint iat-endpiont-v3 exists
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


HttpResponseError: (BadRequest) The request is invalid.
Code: BadRequest
Message: The request is invalid.
Exception Details:	(InferencingClientCallFailed) {"error":{"code":"Validation","message":"{\"errors\":{\"VmSize\":[\"Not enough quota available for Standard_E2s_v3 in SubscriptionId 6bd1f99e-e7cb-4226-b9d5-09433d793bda. Current usage/limit: 0/0. Additional needed: 4 Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-outofquota\"]},\"type\":\"https://tools.ietf.org/html/rfc9110#section-15.5.1\",\"title\":\"One or more validation errors occurred.\",\"status\":400,\"traceId\":\"00-3048b1205a0def7efb9c42e11c7f67c0-db5b0c176d3799b7-01\"}"}}
	Code: InferencingClientCallFailed
	Message: {"error":{"code":"Validation","message":"{\"errors\":{\"VmSize\":[\"Not enough quota available for Standard_E2s_v3 in SubscriptionId 6bd1f99e-e7cb-4226-b9d5-09433d793bda. Current usage/limit: 0/0. Additional needed: 4 Please see troubleshooting guide, available here: https://aka.

In [136]:
import os
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [137]:
%%writefile {deploy_dir}/sample-request.json
{
  "data": [
    {
      "sepal_length": 6.7,
      "sepal_width": 3.1,
      "petal_length": 4.7,
      "petal_width": 1.5
    }
  ]
}

Writing ./deploy/sample-request.json


In [138]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name="shafi1",
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

'"[{\\"predict\\":\\"versicolor\\",\\"setosa\\":0.001944083,\\"versicolor\\":0.9898267126,\\"virginica\\":0.0082292044}]"'

In [140]:
import urllib.request
import json

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
  "data": [
    {
      "sepal_length": 6.7,
      "sepal_width": 3.1,
      "petal_length": 4.7,
      "petal_width": 1.5
    }
  ]
}

body = str.encode(json.dumps(data))

url = 'https://shafi1.centralindia.inference.ml.azure.com/score'
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = '9AsWVOHIV5oIzCeqrRaZu1DxCvcdIrkEeiKAtsMLGirHD4XeCVKtJQQJ99BFAAAAAAAAAAAAINFRAZML2ejo'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Accept': 'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'"[{\\"predict\\":\\"versicolor\\",\\"setosa\\":0.001944083,\\"versicolor\\":0.9898267126,\\"virginica\\":0.0082292044}]"'


In [120]:
from azure.ai.ml import MLClient, load_component

register_model_component = load_component(source="./component/register_model/register_model_component.yml")


In [121]:
ml_client.components.create_or_update(register_model_component)

Uploading register_model (0.11 MBs): 100%|██████████| 109857/109857 [00:00<00:00, 273497.08it/s]




CommandComponent({'latest_version': None, 'intellectual_property': None, 'auto_increment_version': False, 'source': 'REMOTE.WORKSPACE.COMPONENT', 'is_anonymous': False, 'auto_delete_setting': None, 'name': 'register_model', 'description': 'Registers the updated model in Azure ML', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/6bd1f99e-e7cb-4226-b9d5-09433d793bda/resourceGroups/shafi1/providers/Microsoft.MachineLearningServices/workspaces/shafi1/components/register_model/versions/16', 'Resource__source_path': None, 'base_path': '/Users/mdshafiuddin/Desktop/testiat/mlopspoc', 'creation_context': <azure.ai.ml._restclient.v2024_01_01_preview.models._models_py3.SystemData object at 0x13e4cfd00>, 'serialize': <msrest.serialization.Serializer object at 0x13e5c7190>, 'command': 'python register_model_component.py\n  --model_path ${{inputs.model_path}}\n  --model_name ${{inputs.model_name}}\n', 'code': '/subscriptions/6bd1f99e-e7cb-4226-b9d5-09433d793bda/resourceGr

In [122]:
with open("./component/register_model/register_model_component.yml", "r") as f:
    print(f.read())

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json

name: register_model
display_name: Register Model
description: Registers the updated model in Azure ML
version: 16
type: command

inputs:
  model_path:
    type: string
  model_name:
    type: string

outputs:
  registered_model_name:
    type: string
  registered_model_version:
    type: string

code: .

command: >
  python register_model_component.py
    --model_path ${{inputs.model_path}}
    --model_name ${{inputs.model_name}}

environment: azureml:h2o-env:3


In [123]:
deploy_model_component = load_component(source="./component/deploy_model/deploy_model_component.yml")


In [124]:
ml_client.components.create_or_update(deploy_model_component)

Uploading deploy_model (0.0 MBs): 100%|██████████| 4814/4814 [00:00<00:00, 38986.42it/s]




CommandComponent({'latest_version': None, 'intellectual_property': None, 'auto_increment_version': False, 'source': 'REMOTE.WORKSPACE.COMPONENT', 'is_anonymous': False, 'auto_delete_setting': None, 'name': 'deploy_model', 'description': 'Deploys the registered model to an existing endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/6bd1f99e-e7cb-4226-b9d5-09433d793bda/resourceGroups/shafi1/providers/Microsoft.MachineLearningServices/workspaces/shafi1/components/deploy_model/versions/8', 'Resource__source_path': None, 'base_path': '/Users/mdshafiuddin/Desktop/testiat/mlopspoc', 'creation_context': <azure.ai.ml._restclient.v2024_01_01_preview.models._models_py3.SystemData object at 0x13e5e58e0>, 'serialize': <msrest.serialization.Serializer object at 0x13e4e7b80>, 'command': 'python deploy_model_component.py\n  --endpoint_name ${{inputs.endpoint_name}}\n  --environment_name ${{inputs.environment_name}}\n  --environment_version ${{inputs.environment_vers

In [125]:
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default")
print("✅ Authenticated, got token expiring at", token.expires_on)


✅ Authenticated, got token expiring at 1749583344


In [126]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

def ensure_endpoint_exists(ml_client, endpoint_name):
    try:
        ml_client.online_endpoints.get(name=endpoint_name)
        print(f"✅ Endpoint '{endpoint_name}' already exists")
    except Exception:
        print(f"➕ Creating endpoint '{endpoint_name}'")
        ep = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key", description="CI/CD endpoint")
        ml_client.begin_create_or_update(ep).result()
        print(f"✅ Endpoint '{endpoint_name}' created successfully")


In [129]:
ensure_endpoint_exists(ml_client, "shafi1")

➕ Creating endpoint 'shafi1'
✅ Endpoint 'shafi1' created successfully


In [130]:
from azure.identity import DefaultAzureCredential
token = DefaultAzureCredential().get_token("https://management.azure.com/.default")
print("✅ Auth OK, expires:", token.expires_on)


✅ Auth OK, expires: 1749583343


In [131]:
creds = ml_client.online_endpoints.get_credentials(name="shafi1")
print("Primary Key:", creds.primary_key)
print("Secondary Key:", creds.secondary_key)

AttributeError: 'OnlineEndpointOperations' object has no attribute 'get_credentials'

In [133]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

# Try DefaultAzureCredential first (supports MI, SP, CLI)
try:
    credential = DefaultAzureCredential()
    token = credential.get_token("https://management.azure.com/.default")
    print("✅ DefaultAzureCredential acquired token, expires at:", token.expires_on)
except Exception:
    print("⚠️ DefaultAzureCredential failed—falling back to browser authentication")
    credential = InteractiveBrowserCredential()
    token = credential.get_token("https://management.azure.com/.default")
    print("✅ InteractiveBrowserCredential acquired token, expires at:", token.expires_on)

# Initialize MLClient and confirm connection
ml_client = MLClient(
    credential=credential,
    subscription_id="6bd1f99e-e7cb-4226-b9d5-09433d793bda",
    resource_group_name="shafi1",
    workspace_name="shafi1",
)
print("🎉 MLClient initialized and connected to workspace:", ml_client.workspace_name)


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


✅ DefaultAzureCredential acquired token, expires at: 1749618688
🎉 MLClient initialized and connected to workspace: shafi1
